# Introduction to Autokeras

Let's start our journey in autokeras by investigating the classic MNIST dataset

In [1]:
from keras.datasets import mnist
#from autokeras import ImageClassifier
#from autokeras.constant import Constant
from autokeras.image.image_supervised import ImageClassifier, load_image_dataset

Using TensorFlow backend.


Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex.


In [2]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = x_train.reshape(x_train.shape + (1,))
x_test = x_test.reshape(x_test.shape + (1,))

In [4]:
clf = ImageClassifier(verbose=True, augment=False)
clf.fit(x_train, y_train, time_limit=12 * 60 * 60)

Saving Directory: /var/folders/wz/_x09ytx52_q_ry_s7rcg8lg9lxbm17/T/autokeras_17KWSA
Preprocessing the images.
Preprocessing finished.

Initializing search.
Initialization finished.


+----------------------------------------------+
|               Training model 0               |
+----------------------------------------------+
                                                                                                    
No loss decrease after 5 epochs.


Saving model.
+--------------------------------------------------------------------------+
|        Model ID        |          Loss          |      Metric Value      |
+--------------------------------------------------------------------------+
|           0            |  0.24423838332295417   |         0.9776         |
+--------------------------------------------------------------------------+


+----------------------------------------------+
|               Training model 1               |
+----------------------------------

In [ ]:
clf.final_fit(x_train, y_train, x_test, y_test, retrain=True)
y = clf.evaluate(x_test, y_test)
print(y)

Epoch-26, Current Metric - 0.9769:  68%|███████████████       | 320/469 [01:51<00:54,  2.73 batch/s]

Now let's persist our trained model

In [ ]:
clf.export_autokeras_model('mnist_nn_classifier.pkl')

Let's say we want to load our model again at a later time:

In [ ]:
from autokeras.utils import pickle_from_file
model = pickle_from_file('mnist_nn_classifier.pkl')
results = model.evaluate(x_test, y_test)
print(results)

We can also vizualize the model with the help of graphviz (https://graphviz.gitlab.io/download/). To use this specify a path for the model before training.

In [ ]:
clf = ImageClassifier(path=".",verbose=True, augment=False) # Give a custom path of your choice
clf.fit(x_train, y_train, time_limit=30 * 60)
clf.final_fit(x_train, y_train, x_test, y_test, retrain=True)

In [ ]:
visualize('.')

### General CNNsearch

This was easy. Let's give autokeras something somewhat more challenging: the fashion MNIST dataset. This would in principle work the same but let's try to learn a more general Autokeras workflow. The CnnModule can generates neural architecture with basic cnn modules and the ResNet module.

In [ ]:
from tensorflow.examples.tutorials.mnist import input_data
data = input_data.read_data_sets('data/fashion')

In [ ]:
from autokeras import CnnModule
from autokeras.nn.loss_function import classification_loss
from autokeras.nn.metric import Accuracy

TEST_FOLDER = "test"
cnnModule = CnnModule(loss=classification_loss, metric=Accuracy, searcher_args={}, path=TEST_FOLDER, verbose=False)

Note that loss and metric determine the type of training model(classification or regression or others). 
The search_args can be chosen from the following:
        - n_classes: Number of classes in the target classification task.
        - input_shape: Arbitrary, although all dimensions in the input shaped must be fixed.
            Use the keyword argument `input_shape` (tuple of integers, does not include the batch axis)
            when using this layer as the first layer in a model.
        - verbose: Verbosity mode.
        - history: A list that stores the performance of model. Each element in it is a dictionary of 'model_id',
            'loss', and 'metric_value'.
        - neighbour_history: A list that stores the performance of neighbor of the best model.
            Each element in it is a dictionary of 'model_id', 'loss', and 'metric_value'.
        - path: A string. The path to the directory for saving the searcher.
        - metric: An instance of the Metric subclasses.
        - loss: A function taking two parameters, the predictions and the ground truth.
        - generators: A list of generators used to initialize the search.
        - model_count: An integer. the total number of neural networks in the current searcher.
        - descriptors: A dictionary of all the neural network architectures searched.
        - trainer_args: A dictionary. The params for the constructor of ModelTrainer.
        - default_model_len: An integer. Number of convolutional layers in the initial architecture.
        - default_model_width: An integer. The number of filters in each layer in the initial architecture.
        - training_queue: A list of the generated architectures to be trained.
        - x_queue: A list of trained architectures not updated to the gpr.
        - y_queue: A list of trained architecture performances not updated to the gpr.
   
path is the path to store the whole searching process and generated model.
verbose is a boolean. Setting it to true prints to stdout.

Now we can search the architecture:

In [ ]:
cnnModule.fit(n_output_node, input_shape, train_data, test_data, time_limit=24 * 60 * 60)

Here, we have
* train_data: A DataLoader instance representing the training data. 
* test_data: A DataLoader instance representing the testing data. 
* trainer_args: A dictionary containing the parameters of the ModelTrainer constructor. 
* retrain: A boolean of whether reinitialize the weights of the model.

### Other application:

Want to search for a neural network architecture for your structured dataset? You can also have autokeras search for the best Mulitlayer-Perceptron architecture. It works just the same. Hint: the corresponding module is called "MlpModule"

## Exercises

1. Play with the time slots to see if you improve your results.
2. Get yourself a training dataset with very little data and see if you can still get decent performance by having CnnModule to search for an architecture including transfer learning.
3. Compare the modelling time and effort for creating an MNIST classifier in keras directly, without the use of Autokeras.
4. Build an MLP with the corresponding module for the banking dataset. (You can later even evaluate it with the interpretability techniques, we have learned)